In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error ,r2_score
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Admission_Predict.csv')

df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.columns

In [ ]:
df.describe().T

In [ ]:
df["Chance of Admit Binary"] = (df["Chance of Admit "] >= 0.75).astype(int)

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=["Chance of Admit ", "Chance of Admit Binary", "Serial No."])
y = df["Chance of Admit Binary"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled,y_train)

In [ ]:
y_pred = log_model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test,y_pred)

print("Accuracy Score: ",acc)

Modelin doğruluk skoru 0.91. Bu, test verisinin %91’ini doğru sınıflandırdığını gösterir. Yani model genel olarak başarılı bir sınıflandırma yapmaktadır.

In [ ]:
from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)

print("Confusion Matrix: ",conf)

Burada 42 tane "0" değeri doğru bilinmiş, 5 tanesi yanlış tahmin edilmiş. "1" değerinde ise 31 tanesi doğru ,21 tanesi yanlış tahminde bulunmuş.

Model, 42 öğrenciyi doğru şekilde reddetmiş (True Negative) ve 31 öğrenciyi doğru şekilde kabul etmiş (True Positive). Sadece 5 kişiyi yanlış kabul etmiş (False Positive) ve 2 kişiyi yanlış reddetmiş (False Negative). Bu, modelin genelde iyi genelleme yaptığını gösterir.

In [ ]:
from sklearn.metrics import classification_report

clas = classification_report(y_test, y_pred)

print("Classification Report: ",clas)

Sınıf 0 için precision 0.95, yani modelin "reddedildi" tahminlerinin %95’i doğru.  
Sınıf 1 için recall 0.94, yani model kabul edilen öğrencilerin %94’ünü doğru tahmin etmiş.
F1-score değerleri hem sınıf 0 hem de 1 için oldukça yüksektir. Bu, modelin hem doğru tahmin oranının hem de duyarlılığının güçlü olduğunu gösterir.


In [ ]:
feature_names = X.columns  #özellik isimleri

weights = log_model.coef_[0]
bias = log_model.intercept_[0]  #ağırlıklar

for feature, weight in zip(feature_names, weights):  
    print(f"{feature}: {weight:.4f}")               #her özelliğe ait ağırlığı yazdır

print("Bias (intercept):", round(bias, 4))  #bias değeri

Modelin verdiği ağırlıklara göre en etkili değişken CGPA'dır (1.76). GRE skoru (0.72) ve SOP (0.42) gibi değişkenler de kabul olasılığını anlamlı şekilde etkilemektedir. Tüm katsayılar pozitif olduğu için, her bir özelliğin artışı kabul ihtimalini artırmaktadır. Modelin bias (intercept) değeri -0.37’dir, bu da başlangıç eğiliminin düşük olduğunu göstermektedir.


In [ ]:
print("Modelin eğitimde kullandığı iterasyon sayısı:", log_model.n_iter_[0])

LogisticRegression modelinde tol parametresi erken durma için kullanılır. Yani ağırlıklar küçük bir değişimle artık iyileşmiyorsa, eğitim durur.

Bunu kontrol etmek için n_iter_ özelliğini kullanabiliriz.
Eğer bu sayı max_iter=1000'den çok daha azsa, model tolerans sınırına ulaşıp erken durmuş demektir.


Model, sadece 10 iterasyon sonunda tolerans değerine ulaşarak eğitim sürecini tamamladı. Bu durum, modelin çok hızlı bir şekilde optimum değerlere ulaştığını ve daha fazla öğrenmenin gerekli olmadığını gösterir. Bu da modelin veriye hızlı adapte olduğunu gösteren olumlu bir işarettir.

In [ ]:
y_proba = log_model.predict_proba(X_test_scaled)[:, 1]  # sadece '1' sınıfının olasılıkları

y_pred_075 = (y_proba >= 0.75).astype(int)
y_pred_080 = (y_proba >= 0.80).astype(int) #Alternatif eşik değerine göre tahminler

print("Eşik = 0.75")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_075))
print("Classification Report:\n", classification_report(y_test, y_pred_075))# Confusion matrix ve classification report: 0.75

print("Eşik = 0.80")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_080))
print("Classification Report:\n", classification_report(y_test, y_pred_080))# Confusion matrix ve classification report: 0.80


Eşik Değeri Etkisi (Threshold Analysis)
Modelin `0.75` ve `0.80` eşik değerlerinde yaptığı tahminler arasında herhangi bir farklılık gözlemlenmemiştir. Bu durum, modelin tahmin olasılıklarının genellikle 0.75'in çok üstünde veya çok altında yoğunlaştığını gösterir. Yani model sınıflar arasında net bir ayrım yapabilmektedir. Bu da modelin iyi genelleme yaptığına dair olumlu bir işarettir.


### Modeli Geliştirme Önerileri

🔹 **1. Yeni Özellikler Eklenebilir:**  
Veri setinde sadece sayısal sınav puanları ve CGPA gibi özellikler yer almakta. Öğrencinin alanı, başvurduğu bölüm türü, mezun olduğu üniversitenin sıralaması, çalışma deneyimi gibi ek faktörler modele dahil edilirse performans daha da artabilir.

🔹 **2. Eşik Değeri Dinamik Ayarlanabilir:**  
Modelin eşik değeri sabit yerine ROC eğrisi veya Youden’s J istatistiği gibi yöntemlerle optimize edilerek daha iyi sınıflandırma yapılabilir.

🔹 **3. Dengeleme Teknikleri Uygulanabilir:**  
Eğer veri seti daha dengesiz olsaydı (örneğin kabul edilen sayısı çok az/çok fazla), SMOTE veya class weight gibi dengeleme teknikleri denenebilirdi.

🔹 **4. Farklı Sınıflandırma Algoritmaları Kullanılabilir:**  
Lojistik regresyon modeli yerine Random Forest, Support Vector Machines (SVM) gibi algoritmalar kullanılarak doğruluk artırılabilir ve farklı modellerin karşılaştırması yapılabilir.

🔹 **5. Modelin Cross-Validation ile Test Edilmesi:**  
Şu anda veri %80-%20 oranında tek seferde bölünüp test edildi. Ancak k-fold cross-validation uygulanarak daha sağlam ve genel geçer sonuçlara ulaşılabilir.
